<a href="https://colab.research.google.com/github/le-Mon94/HateOffensiveSpeech-Detector-FromAudio/blob/main/Detector_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train Model sklearn linearSVC**

## **Importing Libraries and stuff**

In [1]:
# Import LinearSVC Stuff

import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## **Dataset**

In [2]:
# Importing Dataset

url = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
df = pd.read_csv(url)
del df['Unnamed: 0']

### **Downsampling**

In [3]:
import random

original_offensive_df = df[df['class'] == 1]

# Get the indices of examples to be removed
num_examples_to_remove = len(original_offensive_df) - 1200
indices_to_remove = random.sample(list(original_offensive_df.index), num_examples_to_remove)

# Remove the selected examples
downsampled_offensive_df = original_offensive_df.drop(indices_to_remove)

# Update the original DataFrame with the downsampled class 1 data
df = pd.concat([df[df['class'] != 1], downsampled_offensive_df])

# Shuffle the DataFrame to ensure randomness
df = df.sample(frac=1).reset_index(drop=True)

### **Dataset**

In [4]:
df.head(20)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,Arroz con pollo
1,3,0,0,3,2,@McSheeti means trash lol
2,3,1,0,2,2,@HBergHattie @snkscoyote I wonder if the progs...
3,3,0,0,3,2,RT @GreenLanternJet: I was already 24 when Jet...
4,3,0,0,3,2,RT @CorbynKelly13: Found some more of my origi...
5,3,0,0,3,2,"@JunDishes elissa would call me trash, vile, e..."
6,3,0,0,3,2,remember when the dude who played young ned fo...
7,3,0,2,1,1,RT @TaraTalksMONEY: Tuh! RT @VSamone: When u p...
8,3,0,1,2,2,@danram70 pussy. It's science. Donaire via Rac...
9,3,0,0,3,2,RT @chicagotribune: Chicago will not refund $7...


In [5]:
class_counts = df['class'].value_counts()

print(class_counts)

2    4163
0    1430
1    1200
Name: class, dtype: int64


## **Fitting and Training**

In [6]:
x = df['tweet']
y = df['class']

In [7]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in word_tokenize(nopunc) if word and not re.search(pattern=r"\s+", string=word)]

def split_text(text):
    return text.split()

train_splits = []

for i in tqdm_notebook(range(df.shape[0])):
    train_splits.append(' '.join(process_text(df['tweet'][i])))

  0%|          | 0/6793 [00:00<?, ?it/s]

In [8]:
x = train_splits
y = df['class']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train, x_test, y_train_encoded, y_test_encoded = train_test_split(x, y_encoded, test_size=0.35, random_state=42)

In [9]:
# Define the text classification pipeline

Model = Pipeline([
    ('vect', CountVectorizer(tokenizer=split_text, ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])

# Fit the pipeline to your data

Model.fit(x_train, y_train_encoded)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function split_text at 0x7a2a3c683e20>)),
                ('tfidf', TfidfTransformer()), ('clf', LinearSVC())])

In [10]:
# Output and Training everything

label_mapping = {0: 'hate', 1: 'offensive', 2: 'neither'}

pred_labels_encoded = Model.predict(x_test)

print(x_test)

print(pred_labels_encoded)

pred_labels = label_encoder.inverse_transform(pred_labels_encoded)

submit_df = pd.DataFrame({'text': x_test, 'predicted_label': pred_labels})

submit_df['class'] = submit_df['predicted_label'].map(label_mapping)

submit_df.to_csv('predictions.csv', index=False)

new_df = pd.read_csv('predictions.csv')
new_df.head(20)

['southcarolina has a highschool graduation rate near 50 and thats why they have so many uninformed teabaggers morningjoe', 'Anony386 TurnToLive Tweenki they are wrong you are a nigger always causung mischief among ur own people', 'Your grammar is trash', 'RT joytooawesome imjessicaaaaa vintagemonroe bitch no Im talking to her', 'FattyyTooRaw well Ontario is like kind of a dump now that the ghetto moved here since its cheaper to live here so 128514', 'These brownies httptcoiyqRBn2AMf', 'I hope Charlie brought the lube for this test', 'ennuitimes clever monkey', 'AustinG1135 answer my snapchat faggot butthurt', 'EmmaLouisaSuttl Hit the clam chowderThanks for the wishes and the card', 'RT blogredmachine I wonder if those fans in LA feel like Charlie Brown after Lucy pulls the ball away JoeyVotto', 'RT NEOMD71 Bizarre sports week in Cle Browns win a game that felt like a lossfailed exBrowns QBs duel each other and cavs lose ho8230', 'Watch out ned crc worldcup2014 puravida httptcoDcAgrh4S

,text,predicted_label,class
0,southcarolina has a highschool graduation rate...,2,neither
1,Anony386 TurnToLive Tweenki they are wrong you...,0,hate
2,Your grammar is trash,2,neither
3,RT joytooawesome imjessicaaaaa vintagemonroe b...,1,offensive
4,FattyyTooRaw well Ontario is like kind of a du...,2,neither
5,These brownies httptcoiyqRBn2AMf,2,neither
6,I hope Charlie brought the lube for this test,2,neither
7,ennuitimes clever monkey,2,neither
8,AustinG1135 answer my snapchat faggot butthurt,0,hate
9,EmmaLouisaSuttl Hit the clam chowderThanks for...,2,neither


## **Classification Report**

In [11]:
y_hat = Model.predict(x_train)
report = classification_report(y_train_encoded, y_hat)

conf_matrix = confusion_matrix(y_train_encoded, y_hat)

# Compute accuracy
accuracy = accuracy_score(y_train_encoded, y_hat)

# Print the classification report and other metrics
print("Classification Report:")
print(report)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nAccuracy: {:.4f}".format(accuracy))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       918
           1       1.00      1.00      1.00       813
           2       1.00      1.00      1.00      2684

    accuracy                           1.00      4415
   macro avg       1.00      1.00      1.00      4415
weighted avg       1.00      1.00      1.00      4415


Confusion Matrix:
[[ 918    0    0]
 [   0  813    0]
 [   0    0 2684]]

Accuracy: 1.0000


In [12]:
y_hat = Model.predict(x_test)
report = classification_report(y_test_encoded, y_hat)

conf_matrix = confusion_matrix(y_test_encoded, y_hat)

# Compute accuracy
accuracy = accuracy_score(y_test_encoded, y_hat)

# Print the classification report and other metrics
print("Classification Report:")
print(report)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nAccuracy: {:.4f}".format(accuracy))

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.58      0.67       512
           1       0.72      0.76      0.74       387
           2       0.90      0.97      0.93      1479

    accuracy                           0.85      2378
   macro avg       0.81      0.77      0.78      2378
weighted avg       0.85      0.85      0.84      2378


Confusion Matrix:
[[ 298   89  125]
 [  51  293   43]
 [  22   24 1433]]

Accuracy: 0.8511


## **Export**

In [14]:
import joblib

model_filename = 'trained_detection_model.joblib'
joblib.dump(Model, model_filename)

['trained_detection_model.joblib']